In [17]:
import cv2
import face_recognition
import os
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Input, Model
from keras.layers import Activation, AveragePooling2D, BatchNormalization, Concatenate, Conv2D, Dense, Dropout, Flatten
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Input, Lambda, MaxPooling2D
from keras import backend as K
from tensorflow.keras.applications import Xception

from mtcnn.mtcnn import MTCNN

In [18]:
data_path = r'custom_face_data'

In [19]:
# Create list of paths to images in the custom_face_data directory

imgpaths = []

for dirpath, dirname, filenames in os.walk(data_path):
    for filename in filenames:
        if 'JPG' in filename or 'jpg' in filename or 'jpeg' in filename:
            imgpaths.append(os.path.join(dirpath, filename))

In [20]:
imgpaths = [i for i in imgpaths if "DS_Store" not in i]

# Creates pandas dataframe from the imgpaths list for easy data manipulation
df = pd.DataFrame(imgpaths, columns = ['ImgPath'])

# Gets the identiy of the person from each image path
df['Name'] = df.ImgPath.apply(lambda x: x.split('/')[-2])


In [21]:
input_shape = (96, 96, 3)
embedding_size = 128

# Instantiate Xception Model
xception = Xception(weights="imagenet", input_shape=input_shape, include_top=False)
xception.trainable = False

inputs = Input(shape=input_shape)
# Layer for Xception preprocessing
layer = Lambda(lambda x: (x/127.5)-1)(inputs)
layer = xception(layer)
layer = GlobalMaxPooling2D()(layer)
layer = Dense(embedding_size*4, activation='relu')(layer)
layer = Dense(embedding_size)(layer)
layer = Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(layer)
model = Model(inputs, layer)
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 96, 96, 3)         0         
_________________________________________________________________
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 2048)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_10 (Dense)             (None, 128)               65664     
_________________________________________________________________
lambda_3 (Lambda)            (None, 128)              

In [22]:
weights = r'Triplet Loss/global_max_pooling_FC_4_1_xception.0.4527.hdf5'
model.load_weights(weights)

In [23]:
detector = MTCNN()
def extract_face(imgpath):
    '''Function that reads the path to an image into an an image array with cv2, converting the color to RGB.
    Uses MTCNN to detect faces in an image and to locate the bounding boxes of the face.
    Crops the image to the bounding boxes of the face. 
    Returns a resized version of the cropped image, with shape (96, 96, 3)
    '''
    img = cv2.imread(imgpath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    f = detector.detect_faces(img)
    x1, y1, w, h = f[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2 = abs(x1+w)
    y2 = abs(y1+h)
    face = img[y1:y2, x1:x2]
    face = cv2.resize(face, (96,96))
    return face

def img_to_emb(img_array, embedding_model):
    return embedding_model.predict(np.expand_dims(img_array, axis=0))

In [24]:
# Using the function defined above, get face encodings for every image in database
df['Encodings'] = df.ImgPath.apply(lambda x: img_to_emb(extract_face(x), model))

In [25]:
# Save dataframe to python pickle item for use in live stream face recognition application
df.to_pickle('../FacialRecognition/custom_embeddings_xception.pkl')